导入包

In [1]:
import requests
import json
from bs4 import BeautifulSoup
import time
import warnings

设置(UA,地点,距离),并忽略警告(来自beautifulfoup)

In [2]:
warnings.filterwarnings('ignore')
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36', 
             'Connection': 'keep-alive'}
long1 = 117.224817
latitude1 = 31.825842
distance1 = 6
num1 = 1596451211625
long = 117.235447
latitude = 31.82687
distance = 10
num = 1597242864313

~~获取代理(**已弃用**)~~

In [3]:
def getProxy():
    url = 'http://127.0.0.1:59977/getip?price=0&word=&count=10&type=json&detail=true'
    r = requests.get(url)
    proxys = json.loads(r.text)
    proxyList = []
    for i in proxys:
        print("IP:{}\tPort:{}\tOut IP:{}\tArea:{}".format(i['ip'], i['port'], i['out_ip'], i['area_text']))
        proxyList.append({'http':'http://{}:{}'.format(i['ip'], i['port'])})
    return proxyList

加载cookie(由Chrome插件EditThisCookie导出为json格式)

In [4]:
def loadCookie(fileName):
    cookie = requests.cookies.RequestsCookieJar()
    with open(fileName, 'r') as fp:
        data = json.load(fp)
    for i in data:
        cookie.set(i['name'], i['value'], domain=i['domain'], path=i['path'])
    return cookie

从天眼查获取一定区域内的公司信息(参数从网页请求参数中获取)

In [16]:
def getAllCompany(longtitude, latitude, distance, num, status = None):
    global cookie
    companyNum = 0
    result = []
    with open('{}-{}-{}-{}'.format(longtitude, latitude, distance, num), 'w') as fp:
        count = 0
        for page in range(1, 251):
            if status == None:
                url = 'https://www.tianyancha.com/map/getListByPoint.html?pageNum={}&longtitude={}&latitude={}&distance={}&_={}'.format(page, longtitude, latitude, distance, num)
            else:
                url = 'https://www.tianyancha.com/map/getListByPoint.html?pageNum={}&longtitude={}&latitude={}&distance={}&regStatus={}&_={}'.format(page, longtitude, latitude, distance, status, num)
            while count < 5:
                try:
                    r = requests.get(url, cookies = cookie, headers = headers)
                    count = 0
                    break
                except:
                    print('Retry: {}/5'.format(count))
                    count += 1
            if count == 5:
                raise(Exception, "failed to connect")
            soup = BeautifulSoup(r.text, 'html.parser')
            tags = soup.find_all('td')
            value = []
            count = 0
            for j in tags:
                count += 1
                value.append(j.string)
                if count%8 == 0:
                    #print(allCount, value)
                    result.append({'name':value[1], 'regCurrent':value[3], 'regDate':value[4],\
                                   'type':value[6], 'status':value[7]})
                    value = []
            print('{}/250\tget {} company'.format(page, len(result) - companyNum))
            companyNum = len(result)
        json.dump(result, fp)

~~处理爬取的公司信息(适配于getAllCompany)(**已整合进getAllCompany**)~~

In [6]:
def processHTML(fileName):
    with open(fileName, 'r') as fp:
        allHTML = fp.read()
    HTMLList = allHTML.split('**********')
    allCount = 0
    result = []
    for i in HTMLList:
        soup = BeautifulSoup(i, 'html.parser')
        tags = soup.find_all('td')
        value = []
        count = 0
        for j in tags:
            count += 1
            value.append(j.string)
            if count%8 == 0:
                allCount += 1
                #print(allCount, value)
                result.append({'name':value[1], 'regCurrent':value[3], 'regDate':value[4],\
                               'type':value[6], 'status':value[7]})
                value = []
    return result

~~天眼查版查询具体公司注销信息(**已弃用**)~~

In [7]:
def getCompanyByName(name):
    global cookie#, proxyList
    #proxy = proxyList[randint(0, len(proxyList) - 1)]
    url = 'https://www.tianyancha.com/search?key={}'.format(name)
    r = requests.get(url, cookies = cookie) #proxies = proxy)
    soup = BeautifulSoup(r.text, 'html.parser')
    tags = soup.find_all('a')
    for i in tags:
        #print(i.string)
        if i.string == name:
            #print('find')
            print(i.get('href'))
            url = i.get('href')
            break
    r = requests.get(url, cookies = cookie)#, proxies = proxy)
    print(r.text)
    soup = BeautifulSoup(r.text, 'html.parser')
    tags = soup.find_all('span')#, class_ = 'sec-c3')
    for i in tags:
        print(i)

企查查版查询具体公司注销信息

In [8]:
def getQCCCompanyByName(name):
    global qccCookie
    url = 'http://www.qcc.com/search?key={}'.format(name)
    r = requests.get(url, cookies = qccCookie, headers = headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    tags = soup.find_all('a')
    flag = False
    for i in tags:
        if i.get('href') != None:
            if '/firm/' in i.get('href') and '.shtml' in i.get('href'):
                url = 'https://www.qcc.com' + i.get('href')
                print('Find most closet result: {}'.format(url))
                flag = True
    if flag:
        r = requests.get(url, cookies = qccCookie, headers = headers)
        soup = BeautifulSoup(r.text)
        res = soup.find_all('span', class_ = 'ntag text-danger tooltip-br')
        print(res)
        res = res[0]
        spanTitle = BeautifulSoup(res.get('title'))
        res = spanTitle.find_all('span')
        data = {'date':'', 'reason':''}
        data['date'] = res[0].string[5:]
        data['reason'] = res[1].string[5:]
        return True, data
    else:
        return False, None

~~测试IP(随getProxy弃用)~~

In [9]:
def ipTest():
    global proxyList
    for proxy in proxyList:
        print(proxy)
        r = requests.get('http://icanhazip.com', headers = headers, proxies = proxy)
        print(r.text)

分析注销公司

In [10]:
def resultAnalyze(result):
    companyNum = 0
    notExistNum = 0
    typeNotExistCompany = {}
    typeCompany = {}
    for i in result:
        if i['status'] != '-':
            companyNum += 1
            if i['type'] not in typeCompany.keys():
                typeCompany[i['type']] = 1
            else:
                typeCompany[i['type']] += 1
            if i['status'] != '存续':
                notExistNum += 1
                if i['type'] not in typeNotExistCompany.keys():
                    typeNotExistCompany[i['type']] = 1
                else:
                    typeNotExistCompany[i['type']] += 1
                print("{}-{}-{}-{}-{}".format(i['name'], i['regCurrent'], i['regDate'], i['type'], i['status']))
    print("Not exist rate: {}({}/{})".format(notExistNum/companyNum, notExistNum, companyNum))
    print(typeNotExistCompany)
    print(typeCompany)

In [19]:
cookie = loadCookie('./cookie')